In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import os

Based on the MIMICEL ED dataset description, we also add the lab results that were available to the ED doctors during the patient's time in ED https://physionet.org/content/mimic-iv-ed/2.2/, https://mimic.mit.edu/docs/iv/modules/hosp/labevents/

In [2]:
#Mounting on co-lab


from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/My Drive/XD2Net_CaseStudy') # change folder name




Mounted at /content/drive


In [3]:
my_dir = "/content/drive/My Drive/XD2Net_CaseStudy"
log_file = my_dir+"/MIMICEL ANALYSIS/Data/filtered_mimicel.csv" #the correct and most recent version
lab_file = my_dir+"/MIMICEL ANALYSIS/Data/all_labevents.csv"
labitem_file = my_dir+"/MIMICEL ANALYSIS/Data/d_labitems.csv"

df = pd.read_csv(log_file)
df_lab = pd.read_csv(lab_file)
df_labitem = pd.read_csv(labitem_file)

In [5]:
df['timestamps'] = pd.to_datetime(df['timestamps'])
df['hour'] = df['timestamps'].dt.round("H")

df_lab['storetime'] = pd.to_datetime(df_lab['storetime'])


In [13]:
df_lab.to_csv(lab_file, index = False)

In [21]:
df_labitem.head()

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas


In [16]:
df_lab = df_lab.merge(df_labitem, on='itemid')


In [30]:
#get the top 50% lab test categories with an 'abnormal' flag
lab_cats = df_lab.loc[df_lab['flag'] == 'abnormal'].groupby('label')['stay_id'].size().reset_index().sort_values(by = 'stay_id', ascending = False)
lab_cats['%_tests'] = lab_cats['stay_id'].cumsum()*100/len(df_lab.loc[df_lab['flag'] == 'abnormal'])


In [ ]:
lab_cats['lab_test'] = 'Other'
lab_cats.loc[lab_cats['%_tests'] < 50, 'lab_test'] = lab_cats['label']
lab_cats.head(50)

In [34]:
df_lab = df_lab.merge(lab_cats[['label','lab_test']], on = 'label', how = 'left')
df_lab['lab_test'] = df_lab['lab_test'].fillna('Other')

In [42]:
# Check for non-numeric characters in the 'value' column

non_numeric_pattern = r'[^0-9.]'

df_lab['contains_non_numeric'] = df_lab['value'].astype(str).str.contains(non_numeric_pattern, regex = True)

df_lab.loc[df_lab['contains_non_numeric'] == True].groupby('label')['storetime'].size()

label
% Hemoglobin A1c            1878
24 hr Calcium                  2
24 hr Creatinine               9
25-OH Vitamin D               83
Absolute Basophil Count        4
                           ...  
pCO2                        3309
pH                          5286
pO2                        15795
proBNP, Pleural               19
tacroFK                      401
Name: storetime, Length: 372, dtype: int64

In [44]:
df_lab.loc[df_lab['contains_non_numeric'] == True]

,stay_id,subject_id,specimen_id,storetime,itemid,value,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid,category,lab_test,contains_non_numeric
340,34561576,19304241,29673623,2138-08-12 17:21:00+00:00,51375,___,%,0.0,75.0,NaN,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,True
397,35885015,18603093,43911149,2177-10-20 14:19:00+00:00,51375,___,%,0.0,75.0,NaN,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,True
570,38387976,17991920,28787384,2138-06-12 11:13:00+00:00,51375,___,%,0.0,75.0,NaN,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,True
622,37941321,14846414,57734925,2188-04-17 17:12:00+00:00,51375,___,%,0.0,75.0,NaN,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,True
945,32952086,19994730,40687304,2169-03-23 15:43:00+00:00,51288,___,mm/hr,0.0,15.0,abnormal,Sedimentation Rate,Blood,Hematology,Other,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877581,34372537,19055255,9723886,2169-12-12 15:53:00+00:00,51895,POS,NaN,NaN,NaN,abnormal,"Strep A, Rapid Antigen",Other Body Fluid,Chemistry,Other,True
11877582,30248727,10270170,18634972,2176-04-25 14:58:00+00:00,51309,DONE,NaN,NaN,NaN,NaN,CD14,Bone Marrow,Hematology,Other,True
11877583,34382178,12116839,53246719,2146-09-13 20:40:00+00:00,51423,DONE,NaN,NaN,NaN,NaN,HLA-DR,Other Body Fluid,Hematology,Other,True
11877585,31914217,19674244,68798676,2196-12-30 14:45:00+00:00,50865,___,ug/mL,20.0,25.0,abnormal,Amikacin,Blood,Chemistry,Other,True


In [ ]:
df_lab['valueuom'] = df_lab['valueuom'].fillna("")
df_lab['value_str'] = df_lab['value']+df_lab['valueuom']
df_lab['flag'] = df_lab['flag'].fillna("")
df_lab.loc[df_lab['contains_non_numeric'] == True,'value'] = 0


In [59]:
df_lab = df_lab.loc[df_lab['value'] != '.']
df_lab['value'] = pd.to_numeric(df_lab['value'], errors='coerce').fillna(0)
df_lab.head()


,stay_id,subject_id,specimen_id,storetime,itemid,value,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid,category,lab_test,contains_non_numeric,value_str
0,30989058,13674030,23393037,2146-07-08 21:55:00+00:00,51375,3.0,%,0.0,75.0,,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,False,3%
1,36098916,16658776,97281388,2173-09-10 18:11:00+00:00,51375,54.0,%,0.0,75.0,,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,False,54%
2,31941012,14746942,80768117,2145-08-10 22:19:00+00:00,51375,18.0,%,0.0,75.0,,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,False,18%
3,39627356,11443713,24756867,2146-05-17 00:01:00+00:00,51375,0.0,%,0.0,75.0,,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,False,0%
4,30613875,15901361,54809627,2119-11-02 03:35:00+00:00,51375,3.0,%,0.0,75.0,,Lymphocytes,Joint Fluid,Hematology,Lymphocytes,False,3%


In [63]:
#get the individual lab events by the main lab test categories identified

df_lab_abnormal= df_lab.groupby(['stay_id','storetime'])['flag'].apply(lambda x: (x == 'abnormal').sum()).reset_index(name = 'abnormal_results')


<ipython-input-63-3aaf952d9b4d>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_lab_labels = df_lab.groupby(['stay_id','storetime'])['label','value_str'].agg(join_strings).reset_index()
<ipython-input-63-3aaf952d9b4d>:8: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_lab_labels = df_lab.groupby(['stay_id','storetime'])['label','value_str'].agg(join_strings).reset_index()


In [ ]:

def join_strings(series):
    return ', '.join(series)

df_lab_labels = df_lab.groupby(['stay_id','storetime'])['label'].agg(join_strings).reset_index()


In [79]:
df_lab['value_str'] = df_lab['value_str'].astype(str)
df_lab_values = df_lab.groupby(['stay_id','storetime'])['value_str'].agg(join_strings).reset_index()


In [93]:
df_lab_test= df_lab.groupby(['stay_id','storetime','lab_test'])['flag'].apply(lambda x: (x == 'abnormal').sum()).reset_index(name = 'abnormal_results')


In [94]:
df_lab_test = df_lab_test.pivot(index = ['stay_id','storetime'], columns = 'lab_test', values = 'abnormal_results').reset_index()
df_lab_test

lab_test,stay_id,storetime,Absolute Neutrophil Count,Eosinophils,Glucose,Hematocrit,Hemoglobin,Lymphocytes,MCH,MCHC,Neutrophils,Other,Red Blood Cells,Urea Nitrogen,White Blood Cells
0,30000012,2126-02-14 22:26:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,30000012,2126-02-14 22:36:00+00:00,0.0,1.0,NaN,1.0,1.0,0.0,1.0,0.0,0.0,5.0,1.0,NaN,0.0
2,30000012,2126-02-14 23:04:00+00:00,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN
3,30000012,2126-02-15 00:25:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,30000017,2185-06-18 14:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485242,39999961,2145-05-17 00:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1485243,39999964,2130-06-05 12:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1485244,39999964,2130-06-05 12:43:00+00:00,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0
1485245,39999964,2130-06-05 12:48:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [95]:
df_lab_test = df_lab_test.fillna(0)
df_lab_test['activity'] = 'Lab test result'
df_lab_test = df_lab_test.merge(df_lab_abnormal, on = ['stay_id','storetime'])
df_lab_test = df_lab_test.merge(df_lab_labels, on = ['stay_id','storetime'])
df_lab_test = df_lab_test.merge( df_lab_values, on = ['stay_id','storetime'])


In [96]:
df_lab_test.columns

Index(['stay_id', 'storetime', 'Absolute Neutrophil Count', 'Eosinophils',
       'Glucose', 'Hematocrit', 'Hemoglobin', 'Lymphocytes', 'MCH', 'MCHC',
       'Neutrophils', 'Other', 'Red Blood Cells', 'Urea Nitrogen',
       'White Blood Cells', 'activity', 'abnormal_results', 'label',
       'value_str'],
      dtype='object')

In [97]:
new_cols =[ 'stay_id', 'timestamps', 'Abnormal_ANC', 'Abnormal_Eosinophils',
       'Abnormal_Glucose', 'Abnormal_Hematocrit', 'Abnormal_Hemoglobin', 'Abnormal_Lymphocytes', 'Abnormal_MCH', 'Abnormal_MCHC',
       'Abnormal_Neutrophils', 'Abnormal_Other', 'Abnormal_RBC', 'Abnormal_Urea_Nitrogen',
       'Abnormal_WBC', 'activity', 'Abnormal_all', 'All_tests',
       'All_test_results']

In [98]:
df_lab_test.columns = new_cols
df_lab_test.head()


,stay_id,timestamps,Abnormal_ANC,Abnormal_Eosinophils,Abnormal_Glucose,Abnormal_Hematocrit,Abnormal_Hemoglobin,Abnormal_Lymphocytes,Abnormal_MCH,Abnormal_MCHC,Abnormal_Neutrophils,Abnormal_Other,Abnormal_RBC,Abnormal_Urea_Nitrogen,Abnormal_WBC,activity,Abnormal_all,All_tests,All_test_results
0,30000012,2126-02-14 22:26:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Lab test result,1,Lactate,2.3mmol/L
1,30000012,2126-02-14 22:36:00+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0,0.0,Lab test result,10,"Red Blood Cells, Monocytes, Absolute Basophil ...","2.65m/uL, 16.0%, 0.02K/uL, 55.1%, 2.45K/uL, 1...."
2,30000012,2126-02-14 23:04:00+00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,Lab test result,5,"Alkaline Phosphatase, Glucose, Anion Gap, Urea...","201IU/L, ___mg/dL, 20mEq/L, 36mg/dL, 25mEq/L, ..."
3,30000012,2126-02-15 00:25:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,Lab test result,2,"Epithelial Cells, Yeast, Hyaline Casts, RBC, W...","4#/hpf, NONE, 1#/lpf, 2#/hpf, 25#/hpf, 6.5unit..."
4,30000017,2185-06-18 14:45:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Lab test result,1,Ethanol,___mg/dL


In [99]:
df_lab_test['timestamps']= df_lab_test['timestamps'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [100]:
df_extend = pd.concat([df,df_lab_test]).sort_values(by = ['stay_id','timestamps'], axis=0)
df_extend.head(20)

<ipython-input-100-cd8975c419a5>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_extend = pd.concat([df,df_lab_test]).sort_values(by = ['stay_id','timestamps'], axis=0)


,stay_id,subject_id,hadm_id,timestamps,activity,gender,race,arrival_transport,disposition,seq_num,...,Abnormal_MCH,Abnormal_MCHC,Abnormal_Neutrophils,Abnormal_Other,Abnormal_RBC,Abnormal_Urea_Nitrogen,Abnormal_WBC,Abnormal_all,All_tests,All_test_results
0,30000012,11714491.0,21562392.0,2126-02-14 20:22:00,Enter the ED,F,WHITE,AMBULANCE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,11714491.0,21562392.0,2126-02-14 20:22:01,Triage in the ED,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,30000012,11714491.0,21562392.0,2126-02-14 22:21:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,30000012,11714491.0,21562392.0,2126-02-14 22:22:00,Medicine reconciliation,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
df_extend.to_csv(my_dir+"/MIMICEL ANALYSIS/Data/extended_mimicel.csv", index = False)